

#**Test Notebook - Object Detection**

#**Chen Shalev**





#**Import Libraries**


In [ ]:
import numpy as np
import pandas as pd
import cv2
from sklearn.utils import shuffle
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
import warnings
import os
import json
warnings.simplefilter('ignore')

#**Loading Weights**

In [ ]:
import gdown

# The file ID from the provided Google Drive link
file_id = '1GW3ddn9bc79J0tduVMpBLxFjOtxueIGN'
json_url = f'https://drive.google.com/uc?id={file_id}'
json_output = 'yolov5su.pt'

# Download the file
gdown.download(json_url, json_output, quiet=False)


# Extracting file ID from the Google Drive link
file_url = 'https://drive.google.com/file/d/14L8mYwWXdXLGWp8LbVYLt7OvDNzRw5-o/view?usp=drive_link'
file_id = file_url.split('/')[-2]

# Generating the direct download link
direct_download_url = f'https://drive.google.com/uc?id={file_id}'

# Specifying the output file name
output_file = 'yolov8m.pt'

# Downloading the file
gdown.download(direct_download_url, output_file, quiet=False)

#**My Model**

In [ ]:
!pip install ultralytics

from ultralytics import YOLO
import PIL
from PIL import Image
from IPython.display import display
import os
import pathlib


#**Upload Image**

In [ ]:


# Function to upload file
def upload_file():
    from google.colab import files
    uploaded = files.upload()
    return list(uploaded.keys())[0]  # Return the first uploaded file path

# Initialize YOLO model
model = YOLO("yolov5s.pt")  # Assuming "yolov5s.pt" is the model file you want to use

# Get the file path from the user
new_image_path = upload_file()  # Use the upload_file() function to get the image path

# Predict on the new image
results = model.predict(source=new_image_path, save=True, conf=0.2, iou=0.5)

# Iterate through the results and print object detections
for result in results:
    for box in result.boxes:
        class_id = result.names[box.cls[0].item()]
        cords = box.xyxy[0].tolist()
        cords = [round(x) for x in cords]
        conf = round(box.conf[0].item(), 2)
        print("Object type:", class_id)
        print("Coordinates:", cords)
        print("Probability:", conf)
        print("---")

# Plot the results
plot = results[0].plot()
plot = cv2.cvtColor(plot, cv2.COLOR_BGR2RGB)
display(Image.fromarray(plot))


#**Display Image**

In [ ]:

# Plot the results
plot = results[0].plot()
plot = cv2.cvtColor(plot, cv2.COLOR_BGR2RGB)
display(Image.fromarray(plot))

#**Upload Video**

In [ ]:
# Function to upload file
def upload_file():
    from google.colab import files
    uploaded = files.upload()
    return list(uploaded.keys())[0]  # Return the first uploaded file path

# Initialize YOLO model
model = YOLO("yolov5s.pt")  # Assuming "yolov5s.pt" is the model file you want to use

# Get the file path from the user
new_video_path = upload_file()  # Use the upload_file() function to get the video path

import cv2
from google.colab.patches import cv2_imshow
from IPython.display import HTML
import io
import base64

# Define a color map for different classes
color_map = {
    0: (0, 255, 0),     # Green for class 0
    1: (255, 0, 0),     # Blue for class 1
    2: (0, 0, 255),     # Red for class 2
    3: (255, 255, 0),   # Cyan for class 3
    4: (255, 0, 255),   # Magenta for class 4
    # Add more colors for other classes if needed
}

# Open the video file
cap = cv2.VideoCapture(new_video_path)

# Check if the video opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
else:
    # Get the video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter('output.mp4', fourcc, fps, (width, height))

    # Loop through the video frames
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Predict on the current frame
        results = model.predict(source=frame, save=False, conf=0.2, iou=0.5)

        # Draw detections on the frame
        for result in results:
            for box in result.boxes:
                class_id = box.cls[0].item()
                class_name = result.names[class_id]
                cords = box.xyxy[0].tolist()
                cords = [round(x) for x in cords]
                conf = round(box.conf[0].item(), 2)

                # Get the color for the class
                color = color_map.get(class_id, (0, 255, 255))  # Default color is yellow if class_id not in color_map

                # Draw the bounding box
                cv2.rectangle(frame, (cords[0], cords[1]), (cords[2], cords[3]), color, 2)
                # Put the label
                label = f"{class_name}: {conf}"
                cv2.putText(frame, label, (cords[0], cords[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Write the frame with detections to the output video
        out.write(frame)

    # Release everything when the job is finished
    cap.release()
    out.release()
    cv2.destroyAllWindows()



#**Convert to MKV**






In [ ]:
# delete output.mkv
!rm output.mkv
# Step 1: Install FFmpeg

!apt-get install -y ffmpeg

# Step 2: Convert the MP4 to MKV
!ffmpeg -i output.mp4 output.mkv



#**Display video**

In [ ]:
# Step 3: Display the Video
from IPython.display import HTML
from base64 import b64encode

# Function to display the video
def display_video(file_path):
    video = open(file_path, "rb").read()
    video_encoded = b64encode(video).decode('ascii')
    return HTML(f'''
        <video width="1000" height="800" controls>
            <source src="data:video/mp4;base64,{video_encoded}" type="video/mp4">
        </video>
    ''')

# Display the MKV video
display_video('output.mkv')